In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
num_test_total = 1e6
def read_one_file(file_path, init_iter):
    reward_mean_list = []
    cav_speed_mean_list = []
    reward_normal_mean_list = []
    crash_num_list = []
    reward_crash_mean_list = []
    reward_leave_mean_list = []
    reward_time_mean_list= []
    reward_speedrange_mean_list = []
    reward_lanekeep_mean_list = []
    critical_time_step_number_list = []
    critical_time_step_total_number_list = []
    sub_critical_time_step_number_list = []
    sub_critical_time_step_total_number_list = []
    total_episode_number_list = []
    left_episode_number_list =[]
    total_step_number_list = []
    NDE_crash_rate_list = []
    RHW_list = []
    critical_rss_step_num_list = []
    critical_rss_ratio_list = []
    learner_total_loss_list = []
    learner_policy_loss_list = []
    learner_vf_loss_list = []
    crashnearmiss_traj_pool_size_list = []
    safe2crash_size_list = []
    safe2nearmiss_size_list = []
    total_sampled_crash_list = []
    total_sampled_crash_still_crash_list = []
    eval_original_crash_rate_list = []
    eval_original_nearmiss_crash_rate_list = []
    num_iter = 0
    file_info = {}
    with open(file_path) as f:
        for line in f:
            iter_info = json.loads(line)
            if iter_info["num_healthy_workers"] not in [179,180,480]:
                print(iter_info["num_healthy_workers"])
            num_iter += 1
            reward_mean = iter_info["episode_reward_mean"]
            num_episode = len(iter_info["hist_stats"]["CAV_mean_speed"])
            cav_speed_mean = np.mean(iter_info["hist_stats"]["CAV_mean_speed"])
            reward_normal_mean = np.mean(iter_info["hist_stats"]["reward_normal"])
            crash_num = np.abs(sum(iter_info["hist_stats"]["reward_crash"]))
            reward_crash_mean = np.mean(iter_info["hist_stats"]["reward_crash"])
            reward_leave_mean = np.mean(iter_info["hist_stats"]["reward_leave"])
            reward_time_mean = np.mean(iter_info["hist_stats"]["reward_time"])
            reward_speedrange_mean = np.mean(iter_info["hist_stats"]["reward_speedrange"])
            reward_lanekeep_mean = np.mean(iter_info["hist_stats"]["reward_lanekeep"])
            critical_time_step_number = np.sum(iter_info["hist_stats"]["critical_time_step_number"])
            critical_time_step_number_avg = np.mean(iter_info["hist_stats"]["critical_time_step_number"])
            sub_critical_time_step_number = np.sum(iter_info["hist_stats"]["sub_critical_time_step_number"])
            sub_critical_time_step_number_avg = np.mean(iter_info["hist_stats"]["sub_critical_time_step_number"])
            total_episode_number = np.sum(iter_info["hist_stats"]["total_episode_number"])
            left_episode_number = iter_info["episodes_this_iter"]
            step_number = np.sum(iter_info["hist_stats"]["total_step_number"])
            critical_rss_step_num = np.sum(iter_info["hist_stats"]["critical_time_rss_number"])
            critical_rss_ratio = critical_rss_step_num/critical_time_step_number
            # calculate NDE crash rate
            weighted_result_list = []
            for i in range(len(iter_info["hist_stats"]["total_episode_number"])):
                zero_num = iter_info["hist_stats"]["total_episode_number"][i]
                if iter_info["hist_stats"]["reward_crash"][i] == -1:
                    weighted_result_list.append(iter_info["hist_stats"]["episode_weight"][i])
                    zero_num -= 1
                for j in range(zero_num):
                    weighted_result_list.append(0)
            assert(len(weighted_result_list)==total_episode_number)
            from scipy.stats import norm
            import pandas as pd 
            confidence_interval = 0.1
            z = norm.ppf(1-confidence_interval/2)
            original_result = pd.Series(weighted_result_list)
            # original_result = pd.Series(weighted_result_list)
            crash_mean_result = original_result.rolling(len(original_result), min_periods=1).mean()
            unit_std_result = original_result.rolling(len(original_result), min_periods=1).std()
            half_CI = z*unit_std_result/(np.sqrt(np.array(range(1, len(original_result)+1)))*crash_mean_result)
            # half_CI = z*unit_std_result/(np.sqrt(np.array(range(1, len(original_result)+1)))*1.31e-5)
            half_CI_numpy = half_CI.to_numpy()
            crash_mean_result_numpy = crash_mean_result.to_numpy()
            weighted_result_array = np.array(weighted_result_list)
            learner_total_loss = iter_info["info"]["learner"]["default_policy"]["learner_stats"]["total_loss"]
            learner_policy_loss = iter_info["info"]["learner"]["default_policy"]["learner_stats"]["policy_loss"]
            learner_vf_loss = iter_info["info"]["learner"]["default_policy"]["learner_stats"]["vf_loss"]
            
            try:
#                 print("traj_pool_size",iter_info["hist_stats"]["traj_pool_size"][0])
                crashnearmiss_traj_pool_size = iter_info["hist_stats"]["traj_pool_size"][0][0]
                print(iter_info["hist_stats"]["traj_pool_size"][0])
            except:
                crashnearmiss_traj_pool_size = 0
                
            try:
                safe2crash_size = len(iter_info["evaluation"]["safe2crash_info"])
                safe2nearmiss_size = len(iter_info["evaluation"]["safe2nearmiss_info"])
            except:
                safe2crash_size = 0
                safe2nearmiss_size = 0
               
            try:
                total_sampled_crash_flags = [stats[0] for stats in iter_info["hist_stats"]["interested_crash_sampling_stats"]]
                total_sampled_crash = float(sum(total_sampled_crash_flags))
                total_sampled_crash_still_crash_flags = [stats[1] for stats in iter_info["hist_stats"]["interested_crash_sampling_stats"]]
                total_sampled_crash_still_crash = float(sum(total_sampled_crash_still_crash_flags))
#                 print(total_sampled_crash,total_sampled_crash_still_crash,total_episode_number)
            except:
                total_sampled_crash = total_sampled_crash_still_crash = 0
        
            reward_mean_list.append(reward_mean)
            cav_speed_mean_list.append(cav_speed_mean)
            reward_normal_mean_list.append(reward_normal_mean)
            crash_num_list.append(crash_num)
            reward_crash_mean_list.append(reward_crash_mean)
            reward_leave_mean_list.append(reward_leave_mean)
            reward_time_mean_list.append(reward_time_mean)
            reward_speedrange_mean_list.append(reward_speedrange_mean)
            reward_lanekeep_mean_list.append(reward_lanekeep_mean)
            critical_time_step_number_list.append(critical_time_step_number_avg)
            critical_time_step_total_number_list.append(critical_time_step_number)
            sub_critical_time_step_number_list.append(sub_critical_time_step_number_avg)
            sub_critical_time_step_total_number_list.append(sub_critical_time_step_number)
            total_episode_number_list.append(total_episode_number)
            left_episode_number_list.append(left_episode_number)
            total_step_number_list.append(step_number)
            NDE_crash_rate_list.append(crash_mean_result_numpy[-1])
            RHW_list.append(half_CI_numpy[-1])
            critical_rss_step_num_list.append(critical_rss_step_num)
            critical_rss_ratio_list.append(critical_rss_ratio)
            learner_total_loss_list.append(learner_total_loss)
            learner_policy_loss_list.append(learner_policy_loss)
            learner_vf_loss_list.append(learner_vf_loss)
            crashnearmiss_traj_pool_size_list.append(crashnearmiss_traj_pool_size)
            safe2crash_size_list.append(safe2crash_size)
            safe2nearmiss_size_list.append(safe2nearmiss_size)
            total_sampled_crash_list.append(total_sampled_crash)
            total_sampled_crash_still_crash_list.append(total_sampled_crash_still_crash)
            try:
                eval_original_crash_rate_list.append(iter_info["evaluation"]["original_crash_stats"][0]/sum(iter_info["evaluation"]["original_crash_stats"]))
            except:
                eval_original_crash_rate_list.append(0)
            try:
                eval_original_nearmiss_crash_rate_list.append(iter_info["evaluation"]["original_nearmiss_stats"][0]/sum(iter_info["evaluation"]["original_nearmiss_stats"]))
            except:
                eval_original_nearmiss_crash_rate_list.append(0)
        
    file_info["reward_mean_list"] = reward_mean_list
    file_info["cav_speed_mean_list"] = cav_speed_mean_list    
    file_info["reward_normal_mean_list"] = reward_normal_mean_list
    file_info["crash_num_list"] = crash_num_list
    file_info["reward_crash_mean_list"] = reward_crash_mean_list
    file_info["reward_leave_mean_list"] = reward_leave_mean_list
    file_info["reward_time_mean_list"] = reward_time_mean_list
    file_info["reward_speedrange_mean_list"] = reward_speedrange_mean_list
    file_info["reward_lanekeep_mean_list"] = reward_lanekeep_mean_list
    file_info["critical_time_step_number_list"] = critical_time_step_number_list
    file_info["critical_time_step_total_number_list"] = critical_time_step_total_number_list
    file_info["sub_critical_time_step_number_list"] = sub_critical_time_step_number_list
    file_info["sub_critical_time_step_total_number_list"] = sub_critical_time_step_total_number_list
    file_info["total_episode_number_list"] = total_episode_number_list
    file_info["left_episode_number_list"] = left_episode_number_list
    file_info["total_step_number_list"] = total_step_number_list
    file_info["nde_crash_rate"] = NDE_crash_rate_list
    file_info["rhw_list"] = RHW_list
    file_info["critical_rss_step_num_list"] = critical_rss_step_num_list
    file_info["critical_rss_ratio_list"] = critical_rss_ratio_list
    file_info["learner_total_loss_list"] = learner_total_loss_list
    file_info["learner_policy_loss_list"] = learner_policy_loss_list
    file_info["learner_vf_loss_list"] = learner_vf_loss_list
    file_info["crashnearmiss_traj_pool_size_list"] = crashnearmiss_traj_pool_size_list
    file_info["safe2crash_size_list"] = safe2crash_size_list
    file_info["safe2nearmiss_size_list"] = safe2nearmiss_size_list
    file_info["total_sampled_crash_list"] = total_sampled_crash_list
    file_info["total_sampled_crash_still_crash_list"] = total_sampled_crash_still_crash_list
    file_info["eval_original_crash_rate_list"] = eval_original_crash_rate_list
    file_info["eval_original_nearmiss_crash_rate_list"] = eval_original_nearmiss_crash_rate_list
    return file_info, num_iter

def read_multiple_files(file_path_list, init_iter, split_iter_list = []):
    files_info = {
        "reward_mean_list":[],
        "cav_speed_mean_list":[],
        "reward_normal_mean_list":[],
        "crash_num_list":[],
        "reward_crash_mean_list":[],
        "reward_leave_mean_list":[],
        "reward_time_mean_list":[],
        "reward_speedrange_mean_list":[],
        "reward_lanekeep_mean_list":[],
        "reward_highlycritic_mean_list":[],
        "reward_highlycritic_mean_old_list":[],
        "reward_uncritic_mean_list":[],
        "critical_time_step_number_list":[],
        "critical_time_step_total_number_list":[],
        "sub_critical_time_step_number_list":[],
        "sub_critical_time_step_total_number_list":[],
        "total_episode_number_list":[],
        "left_episode_number_list":[],
        "total_step_number_list":[],
        "nde_crash_rate":[],
        "rhw_list":[],
        "critical_rss_step_num_list":[],
        "critical_rss_ratio_list":[],
        "learner_total_loss_list": [],
        "learner_policy_loss_list": [],
        "learner_vf_loss_list": [],
        "crashnearmiss_traj_pool_size_list": [],
        "safe2crash_size_list": [],
        "safe2nearmiss_size_list": [],
        "total_sampled_crash_list": [],
        "total_sampled_crash_still_crash_list": [],
        "eval_original_crash_rate_list": [],
        "eval_original_nearmiss_crash_rate_list": []
    }
    num_iter = 0
    file_index = 0
    for file_path in file_path_list:
        file_info, num_iter0 = read_one_file(file_path, 1)
        split_num = split_iter_list[file_index]
        if split_num < 0:
            split_num = num_iter0
        files_info["reward_mean_list"].extend(file_info["reward_mean_list"][:split_num])
        files_info["cav_speed_mean_list"].extend(file_info["cav_speed_mean_list"][:split_num])
        files_info["reward_normal_mean_list"].extend(file_info["reward_normal_mean_list"][:split_num])
        files_info["crash_num_list"].extend(file_info["crash_num_list"][:split_num])
        files_info["reward_crash_mean_list"].extend(file_info["reward_crash_mean_list"][:split_num])
        files_info["reward_leave_mean_list"].extend(file_info["reward_leave_mean_list"][:split_num])
        files_info["reward_time_mean_list"].extend(file_info["reward_time_mean_list"][:split_num])
        files_info["reward_speedrange_mean_list"].extend(file_info["reward_speedrange_mean_list"][:split_num])
        files_info["reward_lanekeep_mean_list"].extend(file_info["reward_lanekeep_mean_list"][:split_num])
        files_info["critical_time_step_number_list"].extend(file_info["critical_time_step_number_list"][:split_num])
        files_info["critical_time_step_total_number_list"].extend(file_info["critical_time_step_total_number_list"][:split_num])
        files_info["sub_critical_time_step_number_list"].extend(file_info["sub_critical_time_step_number_list"][:split_num])
        files_info["sub_critical_time_step_total_number_list"].extend(file_info["sub_critical_time_step_total_number_list"][:split_num])
        files_info["total_episode_number_list"].extend(file_info["total_episode_number_list"][:split_num])
        files_info["left_episode_number_list"].extend(file_info["left_episode_number_list"][:split_num])
        files_info["total_step_number_list"].extend(file_info["total_step_number_list"][:split_num])
        files_info["nde_crash_rate"].extend(file_info["nde_crash_rate"][:split_num])
        files_info["rhw_list"].extend(file_info["rhw_list"][:split_num])
        files_info["critical_rss_step_num_list"].extend(file_info["critical_rss_step_num_list"][:split_num])
        files_info["critical_rss_ratio_list"].extend(file_info["critical_rss_ratio_list"][:split_num])
        files_info["learner_total_loss_list"].extend(file_info["learner_total_loss_list"][:split_num])
        files_info["learner_policy_loss_list"].extend(file_info["learner_policy_loss_list"][:split_num])
        files_info["learner_vf_loss_list"].extend(file_info["learner_vf_loss_list"][:split_num])
        files_info["crashnearmiss_traj_pool_size_list"].extend(file_info["crashnearmiss_traj_pool_size_list"][:split_num])
        files_info["safe2crash_size_list"].extend(file_info["safe2crash_size_list"][:split_num])
        files_info["safe2nearmiss_size_list"].extend(file_info["safe2nearmiss_size_list"][:split_num])
        files_info["total_sampled_crash_list"].extend(file_info["total_sampled_crash_list"][:split_num])
        files_info["total_sampled_crash_still_crash_list"].extend(file_info["total_sampled_crash_still_crash_list"][:split_num])
        files_info["eval_original_crash_rate_list"].extend(file_info["eval_original_crash_rate_list"][:split_num])
        files_info["eval_original_nearmiss_crash_rate_list"].extend(file_info["eval_original_nearmiss_crash_rate_list"][:split_num])
        
        num_iter += split_num
        file_index += 1
    return files_info, num_iter

In [ ]:
def plot_training_results(file_info,num_iter,init_iter,settings):
    if "all" in settings:
        fig = plt.figure(figsize=(8,6), dpi=100)
        plt.plot(range(init_iter, init_iter+num_iter), file_info["reward_mean_list"], color="r", label="Total Reward")
        plt.legend()
        ax = fig.gca()
        plt.xlabel("Training Iteration")
        plt.ylabel("Reward")
        plt.legend()
        plt.show()
        
        fig = plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), file_info["cav_speed_mean_list"])
        plt.xlabel("Training Iteration")
        plt.ylabel("CAV mean speed")
        plt.show()
        
        plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), file_info["reward_mean_list"], color="r", alpha=0.3, label="Original")
        df = pd.Series(file_info["reward_mean_list"])
        mean_result = df.rolling(8).mean()
        plt.plot(range(init_iter, init_iter+num_iter), mean_result, alpha=1, label="Smoothed")
        plt.xlabel("Training Iteration")
        plt.ylabel("Total Reward")
        plt.legend()
        plt.show()
        print(file_info["reward_mean_list"].index(max(file_info["reward_mean_list"])))
        
        plt.figure(dpi=100)
        plt.plot(range(init_iter, init_iter+num_iter), file_info["reward_mean_list"], color="r", alpha=0.3)
        df = pd.Series(file_info["reward_mean_list"])
        mean_result = df.rolling(8).mean()
        plt.plot(range(init_iter, init_iter+num_iter), mean_result, color="r")
        plt.xlabel("Training Iteration")
        plt.ylabel("Reward")
        plt.ylim([0.45,0.47])
        plt.title("Total", fontsize=18)
        plt.figure(dpi=100)
        plt.plot(range(init_iter, init_iter+num_iter), file_info["reward_crash_mean_list"], color="orange", alpha=0.3)
        df = pd.Series(file_info["reward_crash_mean_list"])
        mean_result = df.rolling(8).mean()
        plt.plot(range(init_iter, init_iter+num_iter), mean_result, color="orange")
        plt.xlabel("Training Iteration")
        plt.ylabel("Reward")
        plt.title("Crash", fontsize=18)
        
        fig = plt.figure(figsize=(15,24))
        
        plt.subplot(4, 2, 1)
        plt.plot(range(init_iter, init_iter+num_iter), file_info["reward_mean_list"], color="r", alpha=0.3)
        df = pd.Series(file_info["reward_mean_list"])
        mean_result = df.rolling(8).mean()
        plt.plot(range(init_iter, init_iter+num_iter), mean_result, color="r")
        plt.xlabel("Training Iteration")
        plt.ylabel("Reward")
        plt.ylim([0.45,0.47])
        plt.title("Total", fontsize=18)
        
        plt.subplot(4, 2, 2)
        plt.plot(range(init_iter, init_iter+num_iter), file_info["reward_crash_mean_list"], color="orange", alpha=0.3)
        df = pd.Series(file_info["reward_crash_mean_list"])
        mean_result = df.rolling(8).mean()
        plt.plot(range(init_iter, init_iter+num_iter), mean_result, color="orange")
        plt.xlabel("Training Iteration")
        plt.ylabel("Reward")
        plt.title("Crash", fontsize=18)
        
        plt.subplot(4, 2, 3)
        
        plt.subplot(4, 2, 5)
        plt.plot(range(init_iter, init_iter+num_iter), file_info["reward_time_mean_list"], color="m",alpha=0.3)
        df = pd.Series(file_info["reward_time_mean_list"])
        mean_result = df.rolling(8).mean()
        plt.plot(range(init_iter, init_iter+num_iter), mean_result, color="m", label="Driving speed")
        plt.xlabel("Training Iteration")
        plt.ylabel("Reward")
        plt.title("Driving Speed")
        
        plt.subplot(4, 2, 6)
        plt.plot(range(init_iter, init_iter+num_iter), file_info["reward_lanekeep_mean_list"], color="k", alpha=0.3)
        df = pd.Series(file_info["reward_lanekeep_mean_list"])
        mean_result = df.rolling(8).mean()
        plt.plot(range(init_iter, init_iter+num_iter), mean_result, color="k", alpha=0.7, label="Lane keeping")
        plt.xlabel("Training Iteration")
        plt.ylabel("Reward")
        plt.title("Lane keeping")
        
        plt.subplot(4, 2, 7)
        plt.plot(range(init_iter, init_iter+num_iter), file_info["reward_leave_mean_list"], color="g", alpha=0.3)
        df = pd.Series(file_info["reward_leave_mean_list"])
        mean_result = df.rolling(8).mean()
        plt.plot(range(init_iter, init_iter+num_iter), mean_result, color="g", label="Offroad")
        plt.xlabel("Training Iteration")
        plt.ylabel("Reward")
        plt.title("Offroad")
        
        plt.subplot(4, 2, 8)
        plt.plot(range(init_iter, init_iter+num_iter), file_info["reward_speedrange_mean_list"], color="c", alpha=0.3)
        df = pd.Series(file_info["reward_speedrange_mean_list"])
        mean_result = df.rolling(8).mean()
        plt.plot(range(init_iter, init_iter+num_iter), mean_result, color="c", label="Speed limits")
        plt.xlabel("Training Iteration")
        plt.ylabel("Reward")
        plt.title("Speed Limits")

        fig = plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), file_info["cav_speed_mean_list"])
        plt.xlabel("Training Iteration")
        plt.ylabel("CAV mean speed")
        plt.show()
        
        fig = plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), file_info["reward_lanekeep_mean_list"], alpha=0.3, label="Original")
        df = pd.Series(file_info["reward_lanekeep_mean_list"])
        mean_result = df.rolling(8).mean()
        plt.plot(range(init_iter, init_iter+num_iter), mean_result, label="Smoothed")
        plt.xlabel("Training Iteration")
        plt.ylabel("Lane Keeping Reward")
        plt.legend()
        plt.show()
    
    if "crash" in settings:
        crash_reward_df = pd.Series(file_info["reward_crash_mean_list"])
        crash_mean_result = crash_reward_df.rolling(len(crash_reward_df), min_periods=1).mean()
        
        # crash reward
        fig = plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), file_info["reward_crash_mean_list"], color="orange")
        plt.legend(["Crash"])
        plt.xlabel("Training Iteration", fontsize=18)
        plt.ylabel("Reward", fontsize=18)
        plt.yticks(fontsize=13)
        plt.xticks(fontsize=13)
        ax = fig.gca()
        tx = ax.xaxis.get_offset_text()
        tx.set_fontsize(12)
        tx = ax.yaxis.get_offset_text()
        tx.set_fontsize(12)
        plt.show()

        # crash rate
        fig = plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), np.array(file_info["crash_num_list"])/np.array(file_info["total_episode_number_list"]), alpha=0.2)
        crash_rate_df = pd.Series(np.array(file_info["crash_num_list"])/np.array(file_info["total_episode_number_list"]))
        print(crash_rate_df.idxmin(),crash_rate_df[crash_rate_df.idxmin()])
        crash_rate_mean_result = crash_rate_df.rolling(8).mean()
        plt.plot(range(init_iter, init_iter+num_iter), crash_rate_mean_result)
        ax = fig.gca()
        plt.xlabel("Training Iteration", fontsize=16)
        plt.ylabel("Crash Rate during Training", fontsize=16)
        plt.yticks(fontsize=13)
        plt.xticks(fontsize=13)
        ax = fig.gca()
        tx = ax.xaxis.get_offset_text()
        tx.set_fontsize(12)
        tx = ax.yaxis.get_offset_text()
        tx.set_fontsize(12)
        plt.legend(["Original", "Smoothed"])
        plt.show()

        # crash number
        plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), file_info["crash_num_list"], color="orange")
        # ax.set_xticks(np.arange(0, num_iter+1, 4))
        plt.xlabel("Training Iteration")
        plt.ylabel("Crash Number")
        plt.show()
        print(min(file_info["crash_num_list"]), file_info["crash_num_list"].index(min(file_info["crash_num_list"])))
        
        # NDE Crash Rate
        plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), file_info["nde_crash_rate"], color="orange")
        # ax.set_xticks(np.arange(0, num_iter+1, 4))
        plt.ylim(0,0.1)
        plt.xlabel("Training Iteration")
        plt.ylabel("NDE Crash Rate")
        plt.show()
        
        # Final RHW for NDE crash rate estimation
        plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), file_info["rhw_list"], color="orange")
        # ax.set_xticks(np.arange(0, num_iter+1, 4))
        plt.xlabel("Training Iteration")
        plt.ylabel("Final RHW")
        plt.show()
        
        # Size of crash and near-miss trajectory pool
        plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), np.array(file_info["crashnearmiss_traj_pool_size_list"]),alpha=0.2,label="Original")
        crashnearmiss_traj_pool_size_df = pd.Series(file_info["crashnearmiss_traj_pool_size_list"])
        crashnearmiss_traj_pool_size_mean_result = crashnearmiss_traj_pool_size_df.rolling(8).mean()
        plt.plot(range(init_iter, init_iter+num_iter), crashnearmiss_traj_pool_size_mean_result, label="Smoothed")
        # ax.set_xticks(np.arange(0, num_iter+1, 4))
        plt.xlabel("Training Iteration",fontsize=16)
        plt.ylabel("Size of crash and near-miss trajectory pool",fontsize=16)
        plt.yticks(fontsize=13)
        plt.xticks(fontsize=13)
        ax = fig.gca()
        tx = ax.xaxis.get_offset_text()
        tx.set_fontsize(12)
        tx = ax.yaxis.get_offset_text()
        tx.set_fontsize(12)
        plt.legend()
        plt.show()
        
        # Change of crash and near-miss trajectory pool size
        plt.figure()
        crashnearmiss_traj_pool_size_change_list = [file_info["crashnearmiss_traj_pool_size_list"][i+1]-file_info["crashnearmiss_traj_pool_size_list"][i] for i in range(len(file_info["crashnearmiss_traj_pool_size_list"])-1)]
        plt.plot(range(init_iter+1, init_iter+num_iter), np.array(crashnearmiss_traj_pool_size_change_list), color="orange",alpha=0.5,label="Original")
        crashnearmiss_traj_pool_size_diff_df = pd.Series(crashnearmiss_traj_pool_size_change_list)
        crashnearmiss_traj_pool_size_mean_diff_result = crashnearmiss_traj_pool_size_diff_df.rolling(8).mean()
        plt.plot(range(init_iter+1, init_iter+num_iter), crashnearmiss_traj_pool_size_mean_diff_result, label="Smoothed")
        # ax.set_xticks(np.arange(0, num_iter+1, 4))
        plt.xlabel("Training Iteration")
        plt.ylabel("Change of crash and near-miss trajectory pool size")
        plt.legend()
        plt.show()
        
        # Change of safety-critical trajectory pool size
        plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), np.array(file_info["safe2crash_size_list"]),label="Safe2Crash")
        plt.plot(range(init_iter, init_iter+num_iter), np.array(file_info["safe2nearmiss_size_list"]), label="Safe2Nearmiss")
        # ax.set_xticks(np.arange(0, num_iter+1, 4))
        plt.xlabel("Training Iteration")
        plt.ylabel("Change of safety-critical trajectory pool size")
        plt.legend()
        plt.show()
        
        plt.figure()
        new_crash_traj_pool_size_df = pd.Series(np.cumsum(np.array(file_info["safe2crash_size_list"]))+413)
        new_crash_traj_pool_size_mean_result = new_crash_traj_pool_size_df.rolling(8).mean()
        plt.plot(range(init_iter, init_iter+num_iter), np.cumsum(np.array(file_info["safe2crash_size_list"]))+413, color="orange",alpha=0.5,label="Original")
        plt.plot(range(init_iter, init_iter+num_iter), new_crash_traj_pool_size_mean_result, label="Smoothed")
        # ax.set_xticks(np.arange(0, num_iter+1, 4))
        plt.xlabel("Training Iteration")
        plt.ylabel("Number of crash trajectories")
        plt.legend()
        plt.show()
        
        plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), np.cumsum(np.array(file_info["safe2crash_size_list"]))+413, color="orange",alpha=0.5,label="Original")
        plt.plot(range(init_iter, init_iter+num_iter), new_crash_traj_pool_size_mean_result, label="Smoothed")
        # ax.set_xticks(np.arange(0, num_iter+1, 4))
        plt.ylim([1800,1850])
        plt.xlabel("Training Iteration")
        plt.ylabel("Number of crash trajectories")
        plt.legend()
        plt.grid()
        plt.show()
        
        # Number of new crash trajectories
        plt.figure()
        safe2crash_size_list = np.array(file_info["safe2crash_size_list"])
        safe2crash_size_list_df = pd.Series(safe2crash_size_list)
        safe2crash_size_list_mean_result = safe2crash_size_list_df.rolling(8).mean()
        plt.plot(range(init_iter, init_iter+num_iter), safe2crash_size_list, label="Original",color="orange",alpha=0.5)
        plt.plot(range(init_iter, init_iter+num_iter), safe2crash_size_list_mean_result, label="Smoothed",color="blue")
        # ax.set_xticks(np.arange(0, num_iter+1, 4))
        plt.ylim([0,10])
        plt.yticks(range(0,10))
        plt.xlabel("Training Iteration")
        plt.ylabel("Number of new crash trajectories")
        plt.grid()
        plt.legend()
        plt.show()
        
        # Sampled original crash rate
        plt.figure()
        total_sampled_crash_list = np.array(file_info["total_sampled_crash_list"])
        total_sampled_crash_still_crash_list = np.array(file_info["total_sampled_crash_still_crash_list"])
        total_sampled_crash_rate_list = total_sampled_crash_still_crash_list/total_sampled_crash_list
        print(total_sampled_crash_list,total_sampled_crash_still_crash_list,total_sampled_crash_rate_list)
        total_sampled_crash_rate_list_df = pd.Series(total_sampled_crash_rate_list)
        total_sampled_crash_rate_list_mean_result = total_sampled_crash_rate_list_df.rolling(8).mean()
        print(min(total_sampled_crash_rate_list), total_sampled_crash_rate_list.argmin())
        print(total_sampled_crash_list[-1],total_sampled_crash_still_crash_list[-1],total_sampled_crash_rate_list[-1])
        plt.plot(range(init_iter, init_iter+num_iter), total_sampled_crash_rate_list, label="Original",color="orange",alpha=0.5)
        plt.plot(range(init_iter, init_iter+num_iter), total_sampled_crash_rate_list_mean_result, label="Smoothed",color="blue")
        # ax.set_xticks(np.arange(0, num_iter+1, 4))
        plt.ylim([0,1])
        plt.xlabel("Training Iteration")
        plt.ylabel("Sampled original crash rate")
        plt.grid()
        plt.legend()
        plt.show()
        
        # evaluated original crash rate
        plt.figure()
        eval_original_crash_rate_list = np.array(file_info["eval_original_crash_rate_list"])
        eval_original_crash_rate_list_df = pd.Series(eval_original_crash_rate_list)
        eval_original_crash_rate_list_mean_result = eval_original_crash_rate_list_df.rolling(8).mean()
        print(min(eval_original_crash_rate_list), eval_original_crash_rate_list.argmin())
        print(eval_original_crash_rate_list[-1])
        plt.plot(range(init_iter, init_iter+num_iter), eval_original_crash_rate_list, label="Original",color="orange",alpha=0.5)
        plt.plot(range(init_iter, init_iter+num_iter), eval_original_crash_rate_list_mean_result, label="Smoothed",color="blue")
        # ax.set_xticks(np.arange(0, num_iter+1, 4))
        plt.ylim([0,1])
        plt.xlabel("Training Iteration")
        plt.ylabel("Evaluated original crash rate")
        plt.grid()
        plt.legend()
        plt.show()
        
        # evaluated original near-miss crash rate 
        plt.figure()
        eval_original_crash_rate_list = np.array(file_info["eval_original_nearmiss_crash_rate_list"])
        eval_original_crash_rate_list_df = pd.Series(eval_original_crash_rate_list)
        eval_original_crash_rate_list_mean_result = eval_original_crash_rate_list_df.rolling(8).mean()
        print(min(eval_original_crash_rate_list), eval_original_crash_rate_list.argmin())
        print(eval_original_crash_rate_list[-1])
        plt.plot(range(init_iter, init_iter+num_iter), eval_original_crash_rate_list, label="Original",color="orange",alpha=0.5)
        plt.plot(range(init_iter, init_iter+num_iter), eval_original_crash_rate_list_mean_result, label="Smoothed",color="blue")
        # ax.set_xticks(np.arange(0, num_iter+1, 4))
#         plt.ylim([0,1])
        plt.xlabel("Training Iteration")
        plt.ylabel("Evaluated original near-miss crash rate")
        plt.grid()
        plt.legend()
        plt.show()
        
    if "dense" in settings:
        plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), file_info["critical_time_step_number_list"], color="r", label="critical time step")
        plt.plot(range(init_iter, init_iter+num_iter), file_info["sub_critical_time_step_number_list"], color="b", label="slightly critical time step")
#         plt.plot(range(init_iter, init_iter+num_iter), np.array(file_info["sub_critical_time_step_number_list"])+np.array(file_info["critical_time_step_number_list"]), color="g")
        # ax.set_xticks(np.arange(0, num_iter+1, 1))
        plt.xlabel("Training Iteration")
        plt.ylabel("Critical time step")
        plt.legend()
        plt.show()

        plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), file_info["total_episode_number_list"], color="r")
        # ax.set_xticks(np.arange(1, num_iter+1, 1))
        plt.xlabel("Training Iteration")
        plt.ylabel("Total Episode Number")
        plt.show()

        plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), file_info["left_episode_number_list"], color="r")
        # ax.set_xticks(np.arange(1, num_iter+1, 1))
        plt.xlabel("Training Iteration")
        plt.ylabel("Left Episode Number")
        plt.show()

        fig = plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), (np.array(file_info["total_episode_number_list"])-np.array(file_info["left_episode_number_list"]))/np.array(file_info["total_episode_number_list"]), color="r")
        ax = fig.gca()
        # ax.set_xticks(np.arange(0, 71, 5))
        plt.xlabel("Training Iteration")
        plt.ylabel("Episode Skip Proportion")
        plt.show()

        fig = plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), np.array(file_info["left_episode_number_list"])/np.array(file_info["total_episode_number_list"]), color="r")
        ax = fig.gca()
        # ax.set_xticks(np.arange(0, 71, 5))
        plt.xlabel("Training Iteration", fontsize=18)
        plt.ylabel("Episode Remaining Proportion", fontsize=18)
        plt.yticks(fontsize=13)
        plt.xticks(fontsize=13)
        ax = fig.gca()
        tx = ax.xaxis.get_offset_text()
        tx.set_fontsize(12)
        tx = ax.yaxis.get_offset_text()
        tx.set_fontsize(12)
        plt.show()

        fig = plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), (np.array(file_info["critical_time_step_total_number_list"]))/np.array(file_info["total_step_number_list"]), color="r")
        ax = fig.gca()
        # ax.set_xticks(np.arange(0, 71, 5))
        plt.xlabel("Training Iteration")
        plt.ylabel("Step Remaining Proportion")
        plt.legend(["Highly critical time step"])
        plt.show()
        
        fig = plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), file_info["critical_time_step_total_number_list"], color="r")
        ax = fig.gca()
        ymin = int(min(file_info["critical_time_step_total_number_list"])/5000)*5000
        ymax = int(max(file_info["critical_time_step_total_number_list"])/5000+1)*5000
        ax.set_yticks(np.arange(ymin, ymax, 5000))
        plt.xlabel("Training Iteration")
        plt.ylabel("Step Number")
        plt.legend(["Highly critical time step"])
        plt.show()

        fig = plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), (np.array(file_info["sub_critical_time_step_total_number_list"]))/np.array(file_info["total_step_number_list"]), color="r")
        ax = fig.gca()
        # ax.set_xticks(np.arange(0, 71, 5))
        plt.xlabel("Training Iteration")
        plt.ylabel("Step Remaining Proportion")
        plt.legend(["Slightly critical time step"])
        plt.show()

        fig = plt.figure()
        plt.plot(range(init_iter, init_iter+num_iter), (np.array(file_info["critical_time_step_total_number_list"]))/np.array(file_info["total_step_number_list"]), color="r")
        ax = fig.gca()
        # ax.set_xticks(np.arange(0, 71, 5))
        plt.xlabel("Training Iteration", fontsize=18)
        plt.ylabel("Step Remaining Proportion", fontsize=18)
        plt.yticks(fontsize=13)
        plt.xticks(fontsize=13)
        ax = fig.gca()
        tx = ax.xaxis.get_offset_text()
        tx.set_fontsize(12)
        tx = ax.yaxis.get_offset_text()
        tx.set_fontsize(12)
        plt.legend(["Total critical time step"])
        # plt.legend(["total critical time step"])
        plt.show()
        
    


In [ ]:
file_path_list = [
    "/path/to/training/results/which/is/a/json/file"
]
init_iter2 = 370
file_info2, num_iter2 = read_multiple_files(file_path_list, init_iter2, [-1,-1,-1,-1])
settings = ["crash","dense"]
plot_training_results(file_info2,num_iter2,init_iter2,settings)